# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 8:46AM,250649,19,8774225,"Avet Pharmaceuticals Labs, Inc",Released
1,Nov 14 2022 8:44AM,250648,19,8774223,"Emersa Waterbox, LLC",Released
2,Nov 14 2022 8:40AM,250647,19,ACG-2102491855,ACG North America LLC,Released
3,Nov 14 2022 8:37AM,250646,19,ACG-2102491845,ACG North America LLC,Released
4,Nov 14 2022 8:37AM,250646,19,ACG-2102491844,ACG North America LLC,Released
5,Nov 14 2022 8:35AM,250645,10,SOTAH0000831,"Nextsource Biotechnology, LLC",Released
6,Nov 14 2022 8:29AM,250644,19,ACG-2102491849,ACG North America LLC,Executing
7,Nov 14 2022 8:29AM,250644,19,ACG-2102491851,ACG North America LLC,Released
8,Nov 14 2022 7:43AM,250643,19,ADV80007658,"AdvaGen Pharma, Ltd",Released
9,Nov 14 2022 7:43AM,250643,19,ADV80007659,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,250645,Released,1
13,250646,Released,2
14,250647,Released,1
15,250648,Released,1
16,250649,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250645,NaN,1.0
250646,NaN,2.0
250647,NaN,1.0
250648,NaN,1.0
250649,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250634,0.0,2.0
250635,0.0,60.0
250636,0.0,60.0
250637,0.0,38.0
250638,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250634,0,2
250635,0,60
250636,0,60
250637,0,38
250638,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250634,0,2
1,250635,0,60
2,250636,0,60
3,250637,0,38
4,250638,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250634,,2
1,250635,,60
2,250636,,60
3,250637,,38
4,250638,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc"
1,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC"
2,Nov 14 2022 8:40AM,250647,19,ACG North America LLC
3,Nov 14 2022 8:37AM,250646,19,ACG North America LLC
5,Nov 14 2022 8:35AM,250645,10,"Nextsource Biotechnology, LLC"
6,Nov 14 2022 8:29AM,250644,19,ACG North America LLC
8,Nov 14 2022 7:43AM,250643,19,"AdvaGen Pharma, Ltd"
10,Nov 14 2022 7:41AM,250642,19,"Innogenix, LLC"
42,Nov 14 2022 7:36AM,250641,19,"Methapharm, Inc."
56,Nov 14 2022 5:25AM,250638,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",,1
1,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",,1
2,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,,1
3,Nov 14 2022 8:37AM,250646,19,ACG North America LLC,,2
4,Nov 14 2022 8:35AM,250645,10,"Nextsource Biotechnology, LLC",,1
5,Nov 14 2022 8:29AM,250644,19,ACG North America LLC,1,1
6,Nov 14 2022 7:43AM,250643,19,"AdvaGen Pharma, Ltd",,2
7,Nov 14 2022 7:41AM,250642,19,"Innogenix, LLC",14,18
8,Nov 14 2022 7:36AM,250641,19,"Methapharm, Inc.",9,5
9,Nov 14 2022 5:25AM,250638,10,ISDIN Corporation,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",1,
1,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",1,
2,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,1,
3,Nov 14 2022 8:37AM,250646,19,ACG North America LLC,2,
4,Nov 14 2022 8:35AM,250645,10,"Nextsource Biotechnology, LLC",1,
5,Nov 14 2022 8:29AM,250644,19,ACG North America LLC,1,1
6,Nov 14 2022 7:43AM,250643,19,"AdvaGen Pharma, Ltd",2,
7,Nov 14 2022 7:41AM,250642,19,"Innogenix, LLC",18,14
8,Nov 14 2022 7:36AM,250641,19,"Methapharm, Inc.",5,9
9,Nov 14 2022 5:25AM,250638,10,ISDIN Corporation,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",1,
1,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",1,
2,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,1,
3,Nov 14 2022 8:37AM,250646,19,ACG North America LLC,2,
4,Nov 14 2022 8:35AM,250645,10,"Nextsource Biotechnology, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",1,NaN
1,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",1,NaN
2,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,1,NaN
3,Nov 14 2022 8:37AM,250646,19,ACG North America LLC,2,NaN
4,Nov 14 2022 8:35AM,250645,10,"Nextsource Biotechnology, LLC",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",1,0.0
1,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",1,0.0
2,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,1,0.0
3,Nov 14 2022 8:37AM,250646,19,ACG North America LLC,2,0.0
4,Nov 14 2022 8:35AM,250645,10,"Nextsource Biotechnology, LLC",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1503825,166,0.0
19,2005160,31,24.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1503825,166,0.0
1,19,2005160,31,24.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,166,0.0
1,19,31,24.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,166.0
1,19,Released,31.0
2,10,Executing,0.0
3,19,Executing,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19
Status,,
Executing,0.0,24.0
Released,166.0,31.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19
0,Executing,0.0,24.0
1,Released,166.0,31.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19
0,Executing,0.0,24.0
1,Released,166.0,31.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()